In [6]:

candidate = "ds_Anitha_V_Daas"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

plt.rcParams["figure.figsize"] = (8, 4)

os.makedirs(candidate, exist_ok=True)
os.makedirs(f"{candidate}/csv_files", exist_ok=True)
os.makedirs(f"{candidate}/outputs", exist_ok=True)

print("Folder structure created.")

sentiment = pd.read_csv("fear_greed_index.csv")
trades = pd.read_csv("historical_data.csv")
print("Files loaded.")


sentiment.columns = [c.strip() for c in sentiment.columns]

date_col = next((c for c in sentiment.columns if "date" in c.lower()), None)
class_col = next((c for c in sentiment.columns if "fear" in c.lower() or "greed" in c.lower() or "class" in c.lower()), None)

sentiment["Date"] = pd.to_datetime(sentiment[date_col], errors="coerce")
sentiment["Classification"] = sentiment[class_col].astype(str).str.strip().str.title()

sentiment["date"] = sentiment["Date"].dt.normalize()
sentiment["sentiment_num"] = sentiment["Classification"].map({"Fear": 0, "Greed": 1})

sentiment = sentiment[["date", "Classification", "sentiment_num"]].dropna()

print("Sentiment cleaned.")

trades.columns = [c.strip() for c in trades.columns]


time_col = next((c for c in trades.columns if "timestamp" in c.lower() or "time" in c.lower()), "Timestamp")
trades["time"] = pd.to_datetime(trades[time_col], errors="coerce")
trades["date"] = trades["time"].dt.normalize()

pnl_col = "Closed PnL"
size_tokens_col = "Size Tokens"
size_usd_col = "Size USD"

trades[pnl_col] = pd.to_numeric(trades[pnl_col], errors="coerce")
trades[size_tokens_col] = pd.to_numeric(trades[size_tokens_col], errors="coerce")
trades[size_usd_col] = pd.to_numeric(trades[size_usd_col], errors="coerce")

trades["side"] = trades["Side"].astype(str).str.lower().replace({
    "buy": "long",
    "sell": "short"
})

print("Trades cleaned.")


merged = pd.merge(trades, sentiment, on="date", how="left")

merged["Classification"] = merged["Classification"].ffill().bfill()
merged["sentiment_num"] = merged["sentiment_num"].ffill().bfill()

merged.to_csv(f"{candidate}/csv_files/merged_data.csv", index=False)
print("Merge complete!")


plt.bar(merged["Classification"].unique(),
        merged.groupby("Classification")[pnl_col].mean())
plt.title("Average PnL by Sentiment")
plt.ylabel("Mean PnL")
plt.savefig(f"{candidate}/outputs/pnl_by_sentiment.png")
plt.close()

plt.bar(merged["Classification"].unique(),
        merged.groupby("Classification")[size_usd_col].sum())
plt.title("Total USD Volume by Sentiment")
plt.ylabel("Volume (USD)")
plt.savefig(f"{candidate}/outputs/volume_by_sentiment.png")
plt.close()

side = merged.groupby(["Classification", "side"]).size().unstack(fill_value=0)
side.plot(kind="bar", title="Side Distribution (Long/Short) by Sentiment")
plt.tight_layout()
plt.savefig(f"{candidate}/outputs/side_by_sentiment.png")
plt.close()

daily = merged.groupby("date")[pnl_col].mean()
daily.plot(title="Daily Average PnL")
plt.ylabel("PnL")
plt.tight_layout()
plt.savefig(f"{candidate}/outputs/daily_pnl_ts.png")
plt.close()

print("All plots generated successfully!")
print("✔ FINISHED WITHOUT ERRORS ✔")




Folder structure created.
Files loaded.
Sentiment cleaned.
Trades cleaned.
Merge complete!
All plots generated successfully!
✔ FINISHED WITHOUT ERRORS ✔
